In [4]:
import requests
import pandas as pd
from pprint import pprint
import time
import json
import csv
import math
import numpy

In [5]:
state = ['AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA',
         'IA','ID','IL','IN','KS','KY','LA','MA','MD','ME',
         'MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ',
         'NM','NV','NY','OH','OK','OR','PA','RI','SC','SD',
         'TN','TX','UT','VA','VT','WA','WI','WV','WY']

all_city_list = []
for s in state:
    
    purl ="https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldcitiespop&rows=9990&sort=population&refine.country=us&refine.region="+s
    data = requests.get(purl).json()  
    row_cnt = len(data['records'])
    print(s+": "+str(row_cnt))
    for x in range(0, row_cnt):
        all_city_list.append(data['records'][x]['fields'])


# Put all the results in a Pandas DataFrame
all_city_list

AL: 4258
AR: 3158
AZ: 1926
CA: 5436
CO: 1494
CT: 977
DC: 149
DE: 1546
FL: 3158
GA: 5438
IA: 2165
ID: 1212
IL: 3648
IN: 3129
KS: 1613
KY: 4194
LA: 3012
MA: 2176
MD: 7263
ME: 2032
MI: 2718
MN: 2069
MO: 3896
MS: 2503
MT: 1590
NC: 4488
ND: 862
NE: 1169
NH: 965
NJ: 2387
NM: 1446
NV: 612
NY: 5834
OH: 4799
OK: 1863
OR: 1299
PA: 8467
RI: 364
SC: 3513
SD: 913
TN: 4819
TX: 5668
UT: 1933
VA: 6719
VT: 862
WA: 2380
WI: 2303
WV: 3549
WY: 546


[{u'accentcity': u'Birmingham',
  u'city': u'birmingham',
  u'country': u'us',
  u'geopoint': [33.5205556, -86.8025],
  u'latitude': 33.5205556,
  u'longitude': -86.8025,
  u'population': 231621,
  u'region': u'AL'},
 {u'accentcity': u'Montgomery',
  u'city': u'montgomery',
  u'country': u'us',
  u'geopoint': [32.3666667, -86.3],
  u'latitude': 32.3666667,
  u'longitude': -86.3,
  u'population': 198325,
  u'region': u'AL'},
 {u'accentcity': u'Mobile',
  u'city': u'mobile',
  u'country': u'us',
  u'geopoint': [30.6941667, -88.0430556],
  u'latitude': 30.6941667,
  u'longitude': -88.0430556,
  u'population': 190274,
  u'region': u'AL'},
 {u'accentcity': u'Huntsville',
  u'city': u'huntsville',
  u'country': u'us',
  u'geopoint': [34.7302778, -86.5861111],
  u'latitude': 34.7302778,
  u'longitude': -86.5861111,
  u'population': 167528,
  u'region': u'AL'},
 {u'accentcity': u'Tuscaloosa',
  u'city': u'tuscaloosa',
  u'country': u'us',
  u'geopoint': [33.2097222, -87.5691667],
  u'latitude'

In [4]:
df = pd.DataFrame(all_city_list)
df.info()
print(df.columns)
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138520 entries, 0 to 138519
Data columns (total 8 columns):
accentcity    138520 non-null object
city          138520 non-null object
country       138520 non-null object
geopoint      138520 non-null object
latitude      138520 non-null float64
longitude     138520 non-null float64
population    4061 non-null float64
region        138520 non-null object
dtypes: float64(3), object(5)
memory usage: 8.5+ MB
Index([u'accentcity', u'city', u'country', u'geopoint', u'latitude',
       u'longitude', u'population', u'region'],
      dtype='object')


,accentcity,city,country,geopoint,latitude,longitude,population,region
0,Birmingham,birmingham,us,"[33.5205556, -86.8025]",33.520556,-86.802500,231621.0,AL
1,Montgomery,montgomery,us,"[32.3666667, -86.3]",32.366667,-86.300000,198325.0,AL
2,Mobile,mobile,us,"[30.6941667, -88.0430556]",30.694167,-88.043056,190274.0,AL
3,Huntsville,huntsville,us,"[34.7302778, -86.5861111]",34.730278,-86.586111,167528.0,AL
4,Tuscaloosa,tuscaloosa,us,"[33.2097222, -87.5691667]",33.209722,-87.569167,79816.0,AL


In [1]:
df2 = df.dropna(axis=0, subset=['population'])
df2 = df2.sort_values(by=['latitude'], ascending=False)
df2['point_count']=(df2['population'].div(10000))
df2['point_count']=numpy.ceil(df2['point_count']).astype(int)
df2

NameError: name 'df' is not defined

In [6]:
df2.to_csv("popcsv.csv", encoding="utf-8" )

In [23]:
import geojson

def data2geojson(dt):
    features = []
    dt.apply(lambda X: features.append( 
            geojson.Feature(geometry=geojson.Point((X['longitude'], 
                                                    X['latitude'])), 
                properties=dict(name=X['city'], population=X['population'],point_count=X['point_count'] 
                                )))
                                    
    , axis=1)
    with open('map.geojson', 'w') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True)



test1=data2geojson(df2)


In [14]:
df3 = df2[['city', 'geopoint', 'population']].copy()

In [8]:
df3.head()

,city,geopoint,population
0,birmingham,"[33.5205556, -86.8025]",231621.0
1,montgomery,"[32.3666667, -86.3]",198325.0
2,mobile,"[30.6941667, -88.0430556]",190274.0
3,huntsville,"[34.7302778, -86.5861111]",167528.0
4,tuscaloosa,"[33.2097222, -87.5691667]",79816.0


In [86]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4061 entries, 0 to 137990
Data columns (total 3 columns):
city          4061 non-null object
geopoint      4061 non-null object
population    4061 non-null float64
dtypes: float64(1), object(2)
memory usage: 126.9+ KB


In [90]:
data2=df3.to_json(orient='records')
with open('shortpop1.json', 'w') as f:
     f.write((data2))
    
    
# with open('shortpop.json', 'w') as fb:
#     json.dump(data2,fb, indent=4)

AttributeError: 'function' object has no attribute 'pprint'

In [3]:

with open('\\data\\citipop.json', 'w') as f:
    json.dump(all_city_list,f, indent=4)


NameError: name 'json' is not defined

In [ ]:
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [125.6, 10.1]
  },
  "properties": {
    "name": "Dinagat Islands"
  }
}

In [ ]:

import geojson

def data2geojson(df):
    features = []
    df.apply(lambda X: features.append( 
            geojson.Feature(geometry=geojson.Point((X["long"], 
                                                    X["lat"], 
                                                    X["elev"])), 
                properties=dict(name=X["name"], 
                                description=unicode(X["description"].decode('utf8'))))
                                    )
            , axis=1)
    with open('map.geojson', 'w') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True)

col = ['lat','long','elev','name','description']
data = [[-29.9953,-70.5867,760,'A','Place a'],
        [-30.1217,-70.4933,1250,'B','Place b'],
        [-30.0953,-70.5008,1185,'C','Place c']]

df = pd.DataFrame(data, columns=col)

data2geojson(df)
shareimprove this answer:{"type":"Point","coordinates":[-77.21755843075553,38.88128355919233]}}